In [71]:
from manim import *
import manimpango
from common import nuclear
from importlib import reload
import random

In [82]:
%%manim -v WARNING --disable_caching -qm MainScene

reload(nuclear)

config.media_width = "75%"
config.media_embed = True

PARTICLE_SIZE_MULTIPLIER = 0.6
NUCLEON_SEPARATION = 1.0
PARTICLE_LABEL_FONT_SIZE = 30

class MainScene(Scene):
    def construct(self):
        nucleus = nuclear.Nucleus().init_from_nums(12, 12, NUCLEON_SEPARATION, 4, PARTICLE_SIZE_MULTIPLIER, seed=6)

        self.play(*nucleus.create_anims())  # show the nucleus on screen
        
        self.wait()
        # Alpha decay
        daughter1, daughter2, pairs1, pairs2 = nucleus.decay(2, 2, (2, -1), False, False, 1)

        daughter1.shift(4 * RIGHT + 2 * DOWN)
        
        self.play(*map(lambda p: ReplacementTransform(p[0], p[1]), pairs1 + pairs2))

        self.play(daughter2.animate.shift(2 * LEFT))

        self.wait()
        
        # Beta + decay
        # pick a nucleon to decay
        proton = daughter2.find_closest_nucleon((-2, -1), nuclear.PROTON)
        # Turn it into a neutron
        neutron = nuclear.Particle(nuclear.NEUTRON, PARTICLE_SIZE_MULTIPLIER)
        neutron.move_to(proton)

        positron = nuclear.Particle(nuclear.POSITRON, PARTICLE_SIZE_MULTIPLIER)
        positron.move_to(proton)

        beta_plus_label = nuclear.ParticleLabel(positron, PARTICLE_LABEL_FONT_SIZE, r"\beta^+")
        positron_label = nuclear.ParticleLabel(positron, PARTICLE_LABEL_FONT_SIZE)

        self.add(positron, beta_plus_label)
        self.play(Transform(proton, neutron), positron.animate.shift(2 * LEFT + DOWN), run_time=1)

        self.wait()

        self.play(ReplacementTransform(beta_plus_label, positron_label))

        self.wait()
        
        # Gamma decay
        new_daughter2_nucleons = daughter2.get_nucleons_list()
        random.shuffle(new_daughter2_nucleons)
        new_daughter2 = nuclear.Nucleus().init_from_nucleons(new_daughter2_nucleons, NUCLEON_SEPARATION, 4, PARTICLE_SIZE_MULTIPLIER, False)
        new_daughter2.move_to(daughter2)
        # Find some point on the edge of the nucleus
        
        gamma_init_coords = daughter2.find_closest_nucleon((0, 0)).get_center()
        initial_arrow = Arrow(start=gamma_init_coords, end=gamma_init_coords)
        final_arrow = Arrow(start=gamma_init_coords, end=(4 * RIGHT + 2 * UP))
        # Move the gamma particle with the arrow
        gamma_label = MathTex(r"\gamma", font_size=PARTICLE_LABEL_FONT_SIZE)
        gamma_label.move_to(initial_arrow)
        gamma_label.add_updater(lambda x: x.move_to(initial_arrow.point_from_proportion(0.7)).shift(0.3 * UP)))
        self.add(initial_arrow, gamma_label)
        # Go from each original nucleon to the shuffled one
        self.play(ReplacementTransform(initial_arrow, final_arrow), *map(lambda n: ReplacementTransform(n[0], n[1]), zip(new_daughter2_nucleons, new_daughter2.get_nucleons_list())))
        self.wait(1)

SyntaxError: unmatched ')' (<string>, line 64)